In [76]:
import numpy as np
from itertools import product
from bposd.css import css_code

In [77]:
def get_valid_powers_for_summands(summand_combo, l, m, range_A, range_B):
    """
    Generates valid power combinations for a given summand combination,
    respecting the constraints for 'x', 'y', 'z', and the specified ranges for A and B.

    Args:
    - summand_combo: A combination of summands ('x', 'y', 'z').
    - l, m: The limits for 'x' and 'y', respectively. For 'z', the limit is max(l, m).
    - range_A, range_B: Ranges of exponents for terms in A and B to be within.

    Returns:
    - A generator that yields valid combinations of powers for the summands.
    """
    # Define initial power ranges based on summand type
    power_ranges = {
        "x": range(max(1, min(range_A)), min(l, max(range_A))),
        "y": range(max(1, min(range_B)), min(m, max(range_B))),
        "z": range(
            max(1, min(min(range_A), min(range_B))),
            min(max(l, m), max(max(range_A), max(range_B))),
        ),
    }

    # Get the adjusted power range for each summand in the combination
    ranges_for_combo = [power_ranges[summand] for summand in summand_combo]

    # Use product to generate all valid combinations within the specified ranges
    return product(*ranges_for_combo)

def get_net_encoding_rate(
    k: int,
    n: int,
) -> float:
    return k / (2.0 * n)

In [78]:
l, m = 2, 2
weight = 4
power_range_A = [1, 2, 3]
power_range_B = [1, 2, 3]

good_configs = []

In [79]:
I_ell = np.identity(l, dtype=int)
I_m = np.identity(m, dtype=int)
x, y, z = {}, {}, {}

# Generate base matrices x and y
for i in range(l):
        x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
for j in range(m):
    y[j] = np.kron(I_ell, np.roll(I_m, j, axis=1))

# Create base matrix z
for k in range(np.max([l, m])):
    z[k] = np.kron(np.roll(I_ell, k, axis=1), np.roll(I_m, k, axis=1))


In [81]:
weight_A = 2 # Ensure at least one term in A and B # TODO: Could think of also raising to the power of zero leading to identity matrix
weight_B = weight - weight_A

# Generate all combinations of summands in A and B with their respective weights
summands_A = list(product(["x", "y", "z"], repeat=weight_A))
summands_B = list(product(["x", "y", "z"], repeat=weight_B))

for summand_combo_A, summand_combo_B in product(
    summands_A, summands_B
):
    # Check for powers_A
    # Iterate over power ranges for each summand in A and B
    for powers_A in get_valid_powers_for_summands(
        summand_combo_A, l, m, power_range_A, power_range_B
    ):
        # Check for powers_B
        for powers_B in get_valid_powers_for_summands(
            summand_combo_B, l, m, power_range_A, power_range_B
        ):
            A, B = np.zeros((l*m, l*m), dtype=int), np.zeros((l*m, l*m), dtype=int)
            A_poly_sum, B_poly_sum = '', ''

            # Construct A with its summands and powers
            for summand, power in zip(summand_combo_A, powers_A):
                if summand == 'x':
                    matrix = x[power]
                elif summand == 'y':
                    matrix = y[power]
                elif summand == 'z':
                    matrix = z[power]
                A += matrix
                A_poly_sum += f"{summand}{power} + "

            # Construct B with its summands and powers
            for summand, power in zip(summand_combo_B, powers_B):
                if summand == 'x':
                    matrix = x[power]
                elif summand == 'y':
                    matrix = y[power]
                elif summand == 'z':
                    matrix = z[power]
                B += matrix
                B_poly_sum += f"{summand}{power} + "

            A = A % 3
            B = B % 3

            # Remove trailing ' + '
            A_poly_sum = A_poly_sum.rstrip(' + ')
            B_poly_sum = B_poly_sum.rstrip(' + ')

            # Transpose matrices A and B
            AT = np.transpose(A)
            BT = np.transpose(B)
            C = -BT % 3

            # Construct matrices hx and hz
            hx = np.hstack((A, B))
            hz = np.hstack((C, AT))

            # Construct and test the CSS code
            # qcode = css_code(hx, hz)  # Define css_code, assuming it's defined elsewhere
            #if qcode.test():  # Define the test method for qcode
            # r = get_net_encoding_rate(qcode.K, qcode.N)  # Define get_net_encoding_rate
            # encoding_rate_threshold = 1/15
            # if r > encoding_rate_threshold:  # Check your specific criteria for good configurations
            code_config = {
                'l': l,
                'm': m,
                # 'num_phys_qubits': qcode.N,
                # 'num_log_qubits': qcode.K,
                'hx': hx,
                'hz': hz,
                # 'lx': qcode.lx,
                # 'k': qcode.lz.shape[0], 
                # 'encoding_rate': r,
                'A_poly_sum': A_poly_sum,
                'B_poly_sum': B_poly_sum
            }
            good_configs.append(code_config)


In [84]:
len(good_configs)

81

In [85]:
good_configs[11]

{'l': 2,
 'm': 2,
 'hx': array([[0, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 0, 0],
        [0, 1, 1, 0, 1, 1, 0, 0]]),
 'hz': array([[0, 0, 2, 2, 0, 1, 1, 0],
        [0, 0, 2, 2, 1, 0, 0, 1],
        [2, 2, 0, 0, 1, 0, 0, 1],
        [2, 2, 0, 0, 0, 1, 1, 0]]),
 'A_poly_sum': 'x1 + y1',
 'B_poly_sum': 'x1 + z1'}